In [1]:
import numpy as np
from astropy.io import ascii
from astropy.table import Table
from astropy.coordinates import SkyCoord

In [595]:
mastercatfil = './FRB180924.txt'
mastercatfil_nar = './FRB180924_22.0_narrow.txt'

# This file lists already configured files (File -> List -> Allocated in 'configure')
configlistfil = './outlist_FRB180924.txt'

outputsuffix = './FRB180924_2020B_'

#np.random.seed(seed=randseed)


# These are the number of objects to go into the FLD file. For the sky and guide, we want a bit more objects than we will target.
#nobj=2000
#nsky=65 
#nguide=45

##### Read in master catalog ##################
intable = ascii.read(mastercatfil, data_start=6, names=('id', 'rahr', 'ramin','rasec', 'decdeg', 'decmin', 'decsec', 'flag', 'prior', 'rmag', 'dummy1','pm_ra','pm_dec')) 
intable_nar = ascii.read(mastercatfil_nar, data_start=0, names=('id', 'rahr', 'ramin','rasec', 'decdeg', 'decmin', 'decsec', 'flag', 'prior', 'rmag', 'dummy1','pm_ra','pm_dec')) 

getobj = (intable['flag'] == 'P')
getguide = (intable['flag'] == 'F')
getsky = (intable['flag'] == 'S')
getobj_nar = (intable_nar['flag'] == 'P')

objtable = intable[getobj]
guidetable = intable[getguide]
skytable = intable[getsky]
objtable_nar = intable_nar[getobj_nar]

# Make sure that proper motion of guide stars is < 0.015 arcsec/yr
pm_abs = np.sqrt(guidetable['pm_ra']**2  + guidetable['pm_dec']**2)

guidetable = guidetable[pm_abs < 0.015]
###### Finished reading master catalog #######

##### Read list of objects that have already been allocated in other plates ###
listfils = ascii.read(configlistfil, format='commented_header')
confignum = np.size(listfils) # Assume confignum starts at zero
platenum  = int(confignum % 2) # Use modulo 2 to set the plate number
if np.size(listfils) > 0:
    old_objid = []

    for fil in listfils['filname']:
        listtab = ascii.read(fil, data_start=9, format='fixed_width_no_header', 
                             names=('fibernum','objid'), col_starts=(2,6), col_ends=(4,17))
        objid_tmp = listtab['objid']
        old_objid.append(objid_tmp)
        
    old_objid = np.asarray(old_objid)
        
    # Use binary mask to remove objects that have already been allocated from master list
    print('Removing {} objects that have already been configured'.format(np.sum(np.in1d(objtable['id'], old_objid))))
    objtable = objtable[~np.in1d(objtable['id'], old_objid)]


print('Removing {} narrow objects that have already included in narrow file'.format(np.sum(np.in1d(objtable['id'], objtable_nar['id']))))
objtable = objtable[~np.in1d(objtable['id'], objtable_nar['id'])]


#guidetable = np.random.choice(guidetable, size=nguide, replace=False)
#skytable = np.random.choice(skytable, size=nsky, replace=False)


outputfil = 'FRB180924_wide2.txt'
f = open(outputfil, 'w+')

f.write('#Master file of wide 2 field near FRB180924\n'.format(confignum))
f.write('LABEL FRB180924 2df\n')
f.write('UTDATE 2020 10 9\n')
f.write('CENTRE 21 44 25  -40 54 0\n')
f.write('EQUINOX J2000\n')
f.write('\n')

for tab in objtable:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 5, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'])
    f.write(outstr)

for tab in guidetable:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 9, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'])
    f.write(outstr)

for tab in skytable:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 9, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'])
    f.write(outstr)

f.close()


Removing 1424 objects that have already been configured
Removing 0 narrow objects that have already included in narrow file


In [ ]:
#filter narrow field targets which l95>z_frb & remove narrow targets in wide1

#z_191001 = 0.23
#z_180924 = 0.32

In [596]:
photzfil = './photz_F180924_narrow.zout'
photz = ascii.read(photzfil, format='commented_header')

In [535]:
photz

id,z_spec,z_a,z_m1,chi_a,z_p,chi_p,z_m2,odds,l68,u68,l95,u95,l99,u99,nfilt,q_z,z_peak,peak_prob,z_mc
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64
209916192,-1.0,0.561,0.384,0.0245941,0.256,0.290855,0.329,0.949,0.169,0.569,0.095,0.673,0.028,0.767,4,3.17549,0.3287,0.999,0.2449
209915840,-1.0,0.443,0.292,0.03080933,0.333,0.04257344,0.338,0.994,0.181,0.488,0.076,0.572,0.021,0.638,4,0.0264067,0.3376,0.999,0.1522
209915936,-1.0,1.32,1.35,4.024939,0.333,21.87387,0.414,0.536,0.05,1.203,0.02,1.374,0.011,1.452,4,58.7635,0.3267,0.515,0.3517
209916433,-1.0,1.32,1.306,3.498587,1.23,5.409713,1.242,1.0,1.132,1.354,1.064,1.427,0.987,1.459,4,2.5558,1.2396,0.988,1.2498
209916701,-1.0,1.32,1.29,1.879127,1.144,4.469323,1.216,1.0,1.108,1.333,1.046,1.415,0.964,1.458,4,1.58668,1.2149,0.993,1.3507
209916272,-1.0,0.333,0.594,2.177692,0.333,2.177692,0.335,0.98,0.28,0.404,0.059,0.537,0.016,1.229,4,2.69534,0.3318,0.996,0.3231
209914516,-1.0,0.501,0.373,0.04614786,0.231,0.06152331,0.31,1.0,0.223,0.405,0.193,0.509,0.173,0.569,4,0.0878562,0.31,1.0,0.3306
209916730,-1.0,1.462,0.897,2.751304,0.333,3.501276,0.348,0.902,0.211,0.411,0.046,1.152,0.013,1.433,4,7.73139,0.3471,0.999,0.1136
209914026,-1.0,0.256,0.339,0.008570305,0.281,0.01151685,0.354,0.991,0.191,0.535,0.1,0.632,0.034,0.698,4,0.469584,0.3539,0.999,0.4603


In [597]:
mastercatfil_nar = './FRB180924_22.0_narrow.txt'

intable_nar = ascii.read(mastercatfil_nar, data_start=0, names=('id', 'rahr', 'ramin','rasec', 'decdeg', 'decmin', 'decsec', 'flag', 'prior', 'rmag', 'dummy1','pm_ra','pm_dec'))


In [598]:
len(intable_nar)

55

In [599]:
photz = photz[np.in1d(photz['id'], intable_nar['id'])]

In [600]:
len(photz)

55

In [601]:
zfrb = 0.32

In [602]:
photz_8 = photz[np.where((photz['l68']<zfrb))]
photz_7 = photz[np.where((photz['l95']<zfrb) * (photz['l68']>zfrb))]
print(len(photz_8))
print(len(photz_7))

46
4


In [603]:
intable_nar_8 = intable_nar[in_objid_8]
intable_nar_7 = intable_nar[in_objid_7]
print(len(intable_nar_8))
print(len(intable_nar_7))

intable_nar_8 = intable_nar_8[(intable_nar_8['rmag']<21.5)]
intable_nar_7 = intable_nar_7[(intable_nar_7['rmag']<21.5)]
print(len(intable_nar_8))
print(len(intable_nar_7))

46
4
31
1


In [604]:
#mastercatfil = './FRB191001.txt'
mastercatfil_nar = './FRB180924_22.0_narrow.txt'
configlistfil = './outlist_FRB180924.txt'
intable_nar = ascii.read(mastercatfil_nar, data_start=0, names=('id', 'rahr', 'ramin','rasec', 'decdeg', 'decmin', 'decsec', 'flag', 'prior', 'rmag', 'dummy1','pm_ra','pm_dec')) 
getobj_nar = (intable_nar['flag'] == 'P')
objtable_nar = intable_nar[getobj_nar]
listfils = ascii.read(configlistfil, format='commented_header')
if np.size(listfils) > 0:
    old_objid_nar = []

    for fil in listfils['filname']:
        listtab_nar = ascii.read(fil, data_start=9, format='fixed_width_no_header', 
                             names=('fibernum','objid'), col_starts=(2,6), col_ends=(4,17))
        objid_tmp_nar = listtab_nar['objid']
        old_objid_nar.append(objid_tmp_nar)
        
    old_objid_nar = np.asarray(old_objid_nar)

In [605]:
#remove wide1 targets
intable_nar_8 = intable_nar_8[~np.in1d(intable_nar_8['id'], old_objid_nar)]
intable_nar_7 = intable_nar_7[~np.in1d(intable_nar_7['id'], old_objid_nar)]
print(len(intable_nar_8))
print(len(intable_nar_7))

24
1


In [563]:
intable['dummy1'][0]

1

In [606]:
#select p9 targets

prgid =intable['dummy1'][0]
print('program No.'+str(prgid))
gp = [] 

for i in range(len(intable_nar_8)):
    ra = str(intable_nar_8['rahr'][i])+'h'+str(intable_nar_8['ramin'][i])+'m'+str(intable_nar_8['rasec'][i])+'s'
    dec = str(intable_nar_8['decdeg'][i])+'d'+str(intable_nar_8['decmin'][i])+'m'+str(intable_nar_8['decsec'][i])+'s'
    gp.insert(0,ra+' '+dec)
#gp.reverse
g = SkyCoord(gp, frame='icrs')

cen = ["21h33m24s -54d44m54s","21h44m25s -40d54m0s","21h49m23s -52d58m15s","21h49m23s -52d58m15s"]
cp = [cen[prgid]]*len(intable_nar_8)
f = SkyCoord(cp, frame='icrs')

infield = list(g.separation(f).degree)
infield.reverse()

intable_nar_9 = intable_nar_8[np.where(np.array(infield)<0.025)]
intable_nar_8 = intable_nar_8[~np.in1d(intable_nar_8['id'],intable_nar_9['id'])]

print(len(intable_nar_9))
print(len(intable_nar_8))
print(len(intable_nar_7))

program No.1
9
15
1


In [607]:
#write narrow master file

outputfil = 'FRB180924_21.5_narrow_photz_filtered.txt'
f = open(outputfil, 'w+')


f.write('#Master file of narrow field near FRB180924\n')

for tab in intable_nar_9:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f} {13:1d}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 9, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'], 0)
    f.write(outstr)
    
for tab in intable_nar_8:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f} {13:1d}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 8, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'], 0)
    f.write(outstr)
    
for tab in intable_nar_7:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f} {13:1d}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 7, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'], 0)
    f.write(outstr)

f.close()

In [118]:
intable_nar_8

id,rahr,ramin,rasec,decdeg,decmin,decsec,flag,prior,rmag,dummy1,pm_ra,pm_dec
int64,int64,int64,float64,int64,int64,float64,str1,int64,float64,int64,float64,float64
206897915,21,33,8.82,-54,44,55.687,P,9,21.211,0,0.0,0.0
206897953,21,33,11.421,-54,44,58.42,P,9,21.823,0,0.0,0.0
206896982,21,33,14.544,-54,44,6.274,P,9,21.878,0,0.0,0.0
206896366,21,33,20.599,-54,43,35.76,P,9,21.08,0,0.0,0.0
206895580,21,33,31.145,-54,42,51.736,P,9,21.829,0,0.0,0.0
206895776,21,33,18.714,-54,43,4.102,P,9,20.601,0,0.0,0.0
206895995,21,33,13.614,-54,43,15.103,P,9,20.842,0,0.0,0.0
206898016,21,33,8.795,-54,44,59.327,P,9,21.869,0,0.0,0.0
206898040,21,33,28.968,-54,45,4.651,P,9,21.147,0,0.0,0.0


# FLD file generator for FRB191001

In [527]:
randseed = 4023268

# Configuration number, and plate number for the new FLD file
#confignum = 0
#platenum = 0

# Master file for FLD input
mastercatfil = './FRB191001_wide2.txt'
mastercatfil_nar = './FRB191001_21.5_narrow_photz_filtered.txt'

# This file lists already configured files (File -> List -> Allocated in 'configure')
configlistfil = './outlist_FRB191001_wide2.txt'

outputsuffix = './FRB191001_2020B_'

np.random.seed(seed=randseed)


# These are the number of objects to go into the FLD file. For the sky and guide, we want a bit more objects than we will target.
nobj=800
#nsky=65 
#nguide=45

##### Read in master catalog ##################
intable = ascii.read(mastercatfil, data_start=6, names=('id', 'rahr', 'ramin','rasec', 'decdeg', 'decmin', 'decsec', 'flag', 'prior', 'rmag', 'dummy1','pm_ra','pm_dec')) 
intable_nar = ascii.read(mastercatfil_nar, data_start=0, names=('id', 'rahr', 'ramin','rasec', 'decdeg', 'decmin', 'decsec', 'flag', 'prior', 'rmag', 'dummy1','pm_ra','pm_dec','count')) 

##############
prgid =intable['dummy1'][0]
print('program No.'+str(prgid))
gp = [] 
for i in range(len(intable)):
    ra = str(intable['rahr'][i])+'h'+str(intable['ramin'][i])+'m'+str(intable['rasec'][i])+'s'
    dec = str(intable['decdeg'][i])+'d'+str(intable['decmin'][i])+'m'+str(intable['decsec'][i])+'s'
    gp.insert(0,ra+' '+dec)
#gp.reverse
g = SkyCoord(gp, frame='icrs')

cen = ["21h33m24s -54d44m54s","21h44m25s -40d54m0s","21h49m23s -52d58m15s","21h49m23s -52d58m15s"]
cp = [cen[prgid]]*len(intable)
f = SkyCoord(cp, frame='icrs')

infield = list(g.separation(f).degree)
infield.reverse()
intable = intable[np.where(np.array(infield)<1.1)]

###############

getobj = (intable['flag'] == 'P')
getguide = (intable['flag'] == 'F')
getsky = (intable['flag'] == 'S')

objtable = intable[getobj]
guidetable = intable[getguide]
skytable = intable[getsky]

# Make sure that proper motion of guide stars is < 0.015 arcsec/yr
pm_abs = np.sqrt(guidetable['pm_ra']**2  + guidetable['pm_dec']**2)

guidetable = guidetable[pm_abs < 0.015]
###### Finished reading master catalog #######

##### Read list of objects that have already been allocated in other plates ###
listfils = ascii.read(configlistfil, format='commented_header')
confignum = np.size(listfils) # Assume confignum starts at zero
platenum  = int(confignum % 2) # Use modulo 2 to set the plate number
if np.size(listfils) > 0:
    old_objid = []

    for fil in listfils['filname']:
        listtab = ascii.read(fil, data_start=9, format='fixed_width_no_header', 
                             names=('fibernum','objid'), col_starts=(2,6), col_ends=(4,17))
        objid_tmp = listtab['objid']
        old_objid.append(objid_tmp)
        
    old_objid = np.asarray(old_objid)
        
    # Use binary mask to remove objects that have already been allocated from master list
    print('Removing {} wide2 objects that have already been configured'.format(np.sum(np.in1d(objtable['id'], old_objid))))
    objtable = objtable[~np.in1d(objtable['id'], old_objid)]

if np.size(listfils) == 0:
    objtable_nar = intable_nar
    
if np.size(listfils) > 0:
    old_objid = []

    for fil in listfils['filname']:
        listtab = ascii.read(fil, data_start=9, format='fixed_width_no_header', 
                             names=('fibernum','objid','prior'), col_starts=(2,6,48), col_ends=(4,17,49))
        #####
        deltab = []
        for i in range(len(listtab)):
            if len(listtab['objid'][i])!=9:
                deltab.append(i)
        del listtab[deltab]
        #####
        objid_tmp = listtab[np.in1d(listtab['prior'],[7,8,9])]['objid']
        old_objid = list(old_objid)+list(objid_tmp)
        
    old_objid = np.asarray(old_objid,dtype=int)
    for iid in old_objid:
        iin = np.where(intable_nar['id']==iid)
        intable_nar[iin[0][0]]['count'] += 1
        
    getobj_nar = (intable_nar['count']!=2)
    getconf_nar = (intable_nar['count']!=0)
    objtable_nar = intable_nar[getobj_nar]
    print('Removing {} narrow objects that have already been configured'.format(np.sum(~getobj_nar)))
    print('Counting {} narrow objects that have been configured'.format(np.sum(getconf_nar)))

if np.size(objtable) > nobj:
    objtable = np.random.choice(objtable, size=nobj, replace=False)   
    


#guidetable = np.random.choice(guidetable, size=nguide, replace=False)
#skytable = np.random.choice(skytable, size=nsky, replace=False)


outputfil = outputsuffix+'conf{:02d}_p{:1d}.fld'.format(confignum, platenum) 
f = open(outputfil, 'w+')

f.write('#FLD file number {} of field near FRB191001\n'.format(confignum))
f.write('LABEL FRB191001 2df\n')
f.write('UTDATE 2020 10 9\n')
f.write('CENTRE 21 33 24 -54 44 54\n')
f.write('EQUINOX J2000\n')
f.write('\n')

for tab in objtable:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 4, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'])
    f.write(outstr)

for tab in guidetable:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 9, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'])
    f.write(outstr)

for tab in skytable:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 9, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'])
    f.write(outstr)
    
for tab in objtable_nar:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], tab['prior'], tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'])
    f.write(outstr)

f.close()


program No.0
Removing 788 wide2 objects that have already been configured
Removing 35 narrow objects that have already been configured
Counting 36 narrow objects that have been configured


In [334]:
np.where((intable_nar['rmag']<20.3) * (intable_nar['prior']==9))[0].shape

(9,)

In [410]:
prgid =intable['dummy1'][0]
print('program No.'+str(prgid))
gp = [] 

for i in range(len(intable_nar_8)):
    ra = str(intable_nar_8['rahr'][i])+'h'+str(intable_nar_8['ramin'][i])+'m'+str(intable_nar_8['rasec'][i])+'s'
    dec = str(intable_nar_8['decdeg'][i])+'d'+str(intable_nar_8['decmin'][i])+'m'+str(intable_nar_8['decsec'][i])+'s'
    gp.insert(0,ra+' '+dec)
#gp.reverse
g = SkyCoord(gp, frame='icrs')

cen = ["21h33m24s -54d44m54s","21h44m25s -40d54m0s","21h49m23s -52d58m15s","21h49m23s -52d58m15s"]
cp = [cen[prgid]]*len(intable_nar_8)
f = SkyCoord(cp, frame='icrs')

infield = list(g.separation(f).degree)
infield.reverse()
intable_nar_8[np.where(np.array(infield)<0.025)]

program No.0


id,rahr,ramin,rasec,decdeg,decmin,decsec,flag,prior,rmag,dummy1,pm_ra,pm_dec
int64,int64,int64,float64,int64,int64,float64,str1,int64,float64,int64,float64,float64
206896737,21,33,23.946,-54,43,54.181,P,9,20.147,0,0.0,0.0
206896724,21,33,28.173,-54,43,53.717,P,9,20.507,0,0.0,0.0
206898212,21,33,19.468,-54,45,13.518,P,9,20.615,0,0.0,0.0
206897052,21,33,20.455,-54,44,14.971,P,9,19.512,0,0.0,0.0
206897229,21,33,26.003,-54,44,19.907,P,9,20.603,0,0.0,0.0
206896407,21,33,26.225,-54,43,36.764,P,9,20.782,0,0.0,0.0
206898652,21,33,18.958,-54,45,36.99,P,9,21.162,0,0.0,0.0
206897901,21,33,13.727,-54,44,57.757,P,9,20.786,0,0.0,0.0
206898006,21,33,26.271,-54,45,15.044,P,9,19.48,0,0.0,0.0


In [519]:
deltab = []
for i in range(len(listtab)):
    if len(listtab['objid'][i])!=9:
        deltab.append(i)
del listtab[deltab]

# FLD file generator for FRB180924

In [616]:
randseed = 4023268

# Configuration number, and plate number for the new FLD file
#confignum = 0
#platenum = 0

# Master file for FLD input
mastercatfil = './FRB180924_wide2.txt'
mastercatfil_nar = './FRB180924_21.5_narrow_photz_filtered.txt'

# This file lists already configured files (File -> List -> Allocated in 'configure')
configlistfil = './outlist_FRB180924_wide2.txt'

outputsuffix = './FRB180924_2020B_w2_'

np.random.seed(seed=randseed)


# These are the number of objects to go into the FLD file. For the sky and guide, we want a bit more objects than we will target.
nobj=800
nartimes = 2
#nsky=65 
#nguide=45

##### Read in master catalog ##################
intable = ascii.read(mastercatfil, data_start=6, names=('id', 'rahr', 'ramin','rasec', 'decdeg', 'decmin', 'decsec', 'flag', 'prior', 'rmag', 'dummy1','pm_ra','pm_dec')) 
intable_nar = ascii.read(mastercatfil_nar, data_start=0, names=('id', 'rahr', 'ramin','rasec', 'decdeg', 'decmin', 'decsec', 'flag', 'prior', 'rmag', 'dummy1','pm_ra','pm_dec','count')) 

##############
prgid =intable['dummy1'][0]
print('program No.'+str(prgid))
gp = [] 
for i in range(len(intable)):
    ra = str(intable['rahr'][i])+'h'+str(intable['ramin'][i])+'m'+str(intable['rasec'][i])+'s'
    dec = str(intable['decdeg'][i])+'d'+str(intable['decmin'][i])+'m'+str(intable['decsec'][i])+'s'
    gp.insert(0,ra+' '+dec)
#gp.reverse
g = SkyCoord(gp, frame='icrs')

cen = ["21h33m24s -54d44m54s","21h44m25s -40d54m0s","21h49m23s -52d58m15s","21h49m23s -52d58m15s"]
cp = [cen[prgid]]*len(intable)
f = SkyCoord(cp, frame='icrs')

infield = list(g.separation(f).degree)
infield.reverse()
intable = intable[np.where(np.array(infield)<1.05)]

###############

getobj = (intable['flag'] == 'P')
getguide = (intable['flag'] == 'F')
getsky = (intable['flag'] == 'S')

objtable = intable[getobj]
guidetable = intable[getguide]
skytable = intable[getsky]

# Make sure that proper motion of guide stars is < 0.015 arcsec/yr
pm_abs = np.sqrt(guidetable['pm_ra']**2  + guidetable['pm_dec']**2)

guidetable = guidetable[pm_abs < 0.015]
###### Finished reading master catalog #######

##### Read list of objects that have already been allocated in other plates ###
listfils = ascii.read(configlistfil, format='commented_header')
confignum = np.size(listfils) # Assume confignum starts at zero
platenum  = int(confignum % 2) # Use modulo 2 to set the plate number
if np.size(listfils) > 0:
    old_objid = []

    for fil in listfils['filname']:
        listtab = ascii.read(fil, data_start=9, format='fixed_width_no_header', 
                             names=('fibernum','objid'), col_starts=(2,6), col_ends=(4,17))
        objid_tmp = listtab['objid']
        old_objid.append(objid_tmp)
        
    old_objid = np.asarray(old_objid)
        
    # Use binary mask to remove objects that have already been allocated from master list
    print('Removing {} wide2 objects that have already been configured'.format(np.sum(np.in1d(objtable['id'], old_objid))))
    objtable = objtable[~np.in1d(objtable['id'], old_objid)]

if np.size(listfils) == 0:
    objtable_nar = intable_nar
    
if np.size(listfils) > 0:
    old_objid = []

    for fil in listfils['filname']:
        listtab = ascii.read(fil, data_start=9, format='fixed_width_no_header', 
                             names=('fibernum','objid','prior'), col_starts=(2,6,48), col_ends=(4,17,49))
        #####
#        deltab = []
#        for i in range(len(listtab)):
#            if len(listtab['objid'][i])!=9:
#                deltab.append(i)
#        del listtab[deltab]
        #####
        objid_tmp = listtab[np.in1d(listtab['prior'],[7,8,9])]['objid']
        old_objid = list(old_objid)+list(objid_tmp)
        
    old_objid = np.asarray(old_objid,dtype=int)
    for iid in old_objid:
        iin = np.where(intable_nar['id']==iid)
        intable_nar[iin[0][0]]['count'] += 1
        
    getobj_nar = (intable_nar['count']!=nartimes)
    getconf_nar = (intable_nar['count']!=0)
    objtable_nar = intable_nar[getobj_nar]
    print('Removing {} narrow objects that have already been configured'.format(np.sum(~getobj_nar)))
    print('Counting {} narrow objects that have been configured'.format(np.sum(getconf_nar)))

if np.size(objtable) > nobj:
    objtable = np.random.choice(objtable, size=nobj, replace=False)   
    


#guidetable = np.random.choice(guidetable, size=nguide, replace=False)
#skytable = np.random.choice(skytable, size=nsky, replace=False)


outputfil = outputsuffix+'conf{:02d}_p{:1d}.fld'.format(confignum, platenum) 
f = open(outputfil, 'w+')

f.write('#FLD file number {} of field near FRB180924\n'.format(confignum))
f.write('LABEL FRB180924 2df\n')
f.write('UTDATE 2020 10 9\n')
f.write('CENTRE 21 44 25  -40 54 0\n')
f.write('EQUINOX J2000\n')
f.write('\n')

for tab in objtable:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 4, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'])
    f.write(outstr)

for tab in guidetable:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 6, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'])
    f.write(outstr)

for tab in skytable:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], 6, tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'])
    f.write(outstr)
    
for tab in objtable_nar:
    outstr = '{0:12}   {1:02d} {2:02d} {3:06.3f}  {4:+02d} {5:02d} {6:06.3f}   {7:1}   {8:1d}   {9:6.3f}  {10:1d}  {11:+7.4f} {12:+7.4f}\n'.format(tab['id'], tab['rahr'], tab['ramin'], tab['rasec'], tab['decdeg'], tab['decmin'], tab['decsec'], tab['flag'], tab['prior'], tab['rmag'], tab['dummy1'], tab['pm_ra'], tab['pm_dec'])
    f.write(outstr)

f.close()

program No.1
Removing 1859 wide2 objects that have already been configured
Removing 25 narrow objects that have already been configured
Counting 25 narrow objects that have been configured
